In [100]:
import pandas as pd
import numpy as np
import re


# def process_code(code):
#     try:
#         return str(code)[:8]
#     except Exception:
#         return None


def sord(df):
    new_df = pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])

    pd.DataFrame(columns=['time', 'code', 'IM_PQ', 'IM_Q', 'EX_PQ', 'EX_Q'])
    
    # 处理进口金额
    import_amount = df[df['指标'] == '进口金额（美元）']
    import_amount = import_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_PQ'})
    
    # 处理进口量（仅第一数量）
    import_quantity = df[df['指标'] == '进口数量（第一数量）']
    import_quantity = import_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'IM_Q'})
    
    # 处理出口金额
    export_amount = df[df['指标'] == '出口金额（美元）']
    export_amount = export_amount[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_PQ'})
    
    # 处理出口量（仅第一数量）
    export_quantity = df[df['指标'] == '出口数量（第一数量）']
    export_quantity = export_quantity[['时间', '商品', '数值']].rename(columns={'时间': 'time', '商品': 'code', '数值': 'EX_Q'})
    
    # 合并数据
    new_df = (
        import_amount
        .merge(export_amount, on=['time', 'code'], how='outer')
        .merge(import_quantity, on=['time', 'code'], how='outer')
        .merge(export_quantity, on=['time', 'code'], how='outer')
    )
    
    # 填充缺失值为 "空"
    new_df = new_df.fillna(pd.NA)
    new_df.head()
    return new_df


In [101]:
df = pd.read_csv('./09xxxx.cn.csv', encoding='gbk')[:-2]
sorted_df = sord(df)
iv = pd.read_csv('./iv.list.csv')
sorted_df["code"] = sorted_df["code"].str.strip().apply(lambda x: str(x.split()[0]))


In [102]:
# 防止除以零或 NaN 的情况，计算 IM_P
sorted_df["IM_P"] = sorted_df.apply(
    lambda row: row["IM_PQ"] / row["IM_Q"] if row["IM_Q"] not in [0, np.nan] else np.nan, axis=1
)

# 防止除以零或 NaN 的情况，计算 EX_P
sorted_df["EX_P"] = sorted_df.apply(
    lambda row: row["EX_PQ"] / row["IM_Q"] if row["IM_Q"] not in [0, np.nan] else np.nan, axis=1
)

# 从 time 列提取年份并创建 year 列
sorted_df["year"] = sorted_df["time"].str[-4:].astype(int)
# sorted_df = remove2024(sorted_df)

In [103]:
sorted_df['year'] = sorted_df['year'].astype(int)
iv['year'] = iv['year'].astype(int)
df = sorted_df.merge(iv, on='year', how='left')

In [104]:
df["lgdp"] = df["gdp"].apply(np.log)
df["lpop"] = df["pop"].apply(np.log)
df["lfdi"] = df["fdi"].apply(np.log)
df["lexp"] = df['EX_P'].apply(np.log)
df["limp"] = df['IM_P'].apply(np.log)
df["lexq"] = df['EX_Q'].apply(np.log)
df["limq"] = df['IM_Q'].apply(np.log)

In [105]:
df['len'] = df['code'].apply(len)
df['dropped'] = (
        (df["IM_PQ"].isna()) & 
        (df["EX_PQ"].isna()) & 
        (df["IM_Q"].isna()) & 
        (df["EX_Q"].isna())
).astype(int)

df['missing1'] = (
        (df["IM_PQ"].isna()) |
        (df["EX_PQ"].isna()) |
        (df["IM_Q"].isna()) | 
        (df["EX_Q"].isna())
).astype(int)


In [106]:
rate = pd.read_csv('./exc_m.csv', encoding='utf-8')
df = df.merge(rate, on='time', how='left')

In [107]:
df.isin([-np.inf, np.inf]).sum()

time         0
code         0
IM_PQ        0
EX_PQ        0
IM_Q         0
EX_Q         0
IM_P         0
EX_P         0
year         0
fdi          0
gdp          0
pop          0
tea          0
btea         0
gtea         0
lgdp         0
lpop         0
lfdi         0
lexp         0
limp         0
lexq         0
limq        58
len          0
dropped      0
missing1     0
rate         0
dtype: int64

In [108]:
df.replace([np.inf, -np.inf], np.nan, inplace=True)

In [109]:
df['is_tea'] = df['code'].str[:4].isin(['0901', '0902', '0903']).astype(int)
df['i0901'] = df['code'].str[:4].isin(['0901']).astype(int)
df['i0902'] = df['code'].str[:4].isin(['0902']).astype(int)
df['i0903'] = df['code'].str[:4].isin(['0903']).astype(int)

In [113]:
tea_e = pd.read_csv("./tea_sq.csv", encoding='utf-8')
df = df.merge(tea_e, on='year', how='left')

In [114]:
df.to_csv("./09.cn.csv", encoding='utf-8', index=False)


In [115]:
df

,time,code,IM_PQ,EX_PQ,IM_Q,EX_Q,IM_P,EX_P,year,fdi,...,len,dropped,missing1,rate,is_tea,i0901,i0902,i0903,tea_sq,tea_pr
0,01-2017,09011100,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
1,01-2017,09012100,6642370.0,NaN,1897820.0,NaN,3.500000,NaN,2017,13632000.0,...,8,0,1,6.858514,1,1,0,0,2848.72000,246.04000
2,01-2017,09012200,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
3,01-2017,09019010,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
4,01-2017,09019020,NaN,NaN,NaN,NaN,NaN,NaN,2017,13632000.0,...,8,1,1,6.858514,1,1,0,0,2848.72000,246.04000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5854,12-2023,0904,2695.0,NaN,99.0,NaN,27.222222,NaN,2023,16325000.0,...,4,0,1,7.150171,0,0,0,0,3429.95156,354.11234
5855,12-2023,0906,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,4,1,1,7.150171,0,0,0,0,3429.95156,354.11234
5856,12-2023,0909,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,4,1,1,7.150171,0,0,0,0,3429.95156,354.11234
5857,12-2023,0910,NaN,NaN,NaN,NaN,NaN,NaN,2023,16325000.0,...,4,1,1,7.150171,0,0,0,0,3429.95156,354.11234


In [116]:
df["time"].describe()

count        5859
unique         93
top       01-2017
freq           63
Name: time, dtype: object

In [117]:
tea_e

,year,tea_sq,tea_pr
0,2017,2848.72000,246.04000
1,2018,2985.80000,261.03931
2,2019,3104.76557,277.72043
3,2020,3217.00000,293.20000
4,2021,3307.84107,316.39612
5,2022,3392.70508,334.20530
6,2023,3429.95156,354.11234
